In [1]:
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag

import requests
import json
import os

def getInfoDicFromJavhoo(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("img", class_="alignnone size-full")['src']
    fullTitle = bsSoup.find("h1", class_="h3-size entry-title").get_text()
    title = fullTitle[fullTitle.find(' ')+1:] 
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", class_="project_info")  
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :')
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                else:
                    body = ""
                valueDic[header] = body
                p = p.find_next_sibling('p')
        else:
            header = p.get_text().strip(' :')
            body = []
            p = p.find_next_sibling('p')
            if (p.contents[0]['class'] == ['genre']):
                 for child in p.contents[0]:
                    body.append(child.get_text())
            p = p.find_next_sibling('p')
            valueDic[header] = body

    valueDic['網站'] = 'Javhoo'
    return valueDic


def getDicFromJavhoo(avID):
    url = "https://www.javhoo.com/av/" + avID.lower()
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavhoo(soup)



def getInfoDicFromJavBus(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("a", {'class': "bigImage"}).find('img')['src']
    fullTitle = bsSoup.find("div", {'class': "container"}).find('h3').get_text()
    title = fullTitle[fullTitle.find(' ')+1:]  # title might contains actress name, need to remove it later
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", {'class': "col-md-3 info"})   
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :\n')
#                print('header is: ', header)
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                elif (len(p) == 3):
                    body = p.contents[2].get_text() # try&error, the last element contains data
                else:
                    body = ""
#                print('body is: ', body)
                if header == '識別碼':
                    header = '番號'  # change JavBus naming to Javhoo naming
                valueDic[header] = body.strip(' :\n')
                p = p.find_next_sibling('p')
        elif p.has_attr('class') and p['class'] == ['header']:
            # parsing genre tag for JavBus
            header = p.get_text().strip(' :\n')
#            print('header2 is: ', header)
            body = [] 
            p = p.find_next_sibling('p')
            for item in p:
                if not isinstance(item, NavigableString) and item.has_attr('class') and item['class'] == ['genre']:
                    body.append(item.get_text().strip(' :\n'))           
#            print('body2 is: ', body)
            valueDic[header] = body
            p = p.find_next_sibling('p')            
        elif p.has_attr('class') and p['class'] == ['star-show']:
            # parsing actor for JavBus
            header = p.get_text().strip(' :\n')
#            print('header3 is: ', header)
            body = [] 
            p = p.find_next_sibling('p')
            if p:
                for item in p:            
                    if not isinstance(item, NavigableString) and item.has_attr('class') and item['class'] == ['genre']:
                        body.append(item.get_text().strip(' :\n'))
                p = p.find_next_sibling('p')  
#            print('body3 is: ', body)
            valueDic[header] = body

    valueDic['網站'] = 'JavBus'
    return valueDic


def getDicFromJavBus(avID):
    url = "https://www.javbus.com/" + avID.upper()
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavBus(soup)






def downloadImage(webPath, avID):
    ext = webPath[webPath.rfind('.'):]
    filename = localFolder + '/' + avID + ext
    r = requests.get(webPath, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)  

def dumpDic2Json(valueDic):
    ID = valueDic['序號']
    filename = localFolder + '/' + ID +'.json'
    f = open(filename, "w")
    json.dump(valueDic, f)
    f.close()
    
def loadDicFromJson(avID):
    filename = localFolder + '/' + avID +'.json'    
    f = open(filename, 'r')
    valueDic = json.load(f)
    f.close()
    return valueDic
    
def checkAvIdFileExist(avID):
    filename = localFolder + '/' + avID +'.json'
    if os.path.exists(filename):
        return True
    return False

def checkAvIdImgExist(avID):
    jpg = localFolder + '/' + avID +'.jpg'
    png = localFolder + '/' + avID +'.png'
    if os.path.exists(jpg) or os.path.exists(png):
        return True
    return False


def completeAvID(avID, itemResult, showInfo = False):
    vDic = {}
    #AvId status, 1 not download before, 2 not exist on Javhoo, 3 downloaded
    if not checkAvIdFileExist(avID):
        vDic = getDicFromJavhoo(avID)
        if (len(vDic) == 0):
            itemResult[0] = 404
            vDic = getDicFromJavBus(avID)
            if (len(vDic) == 0):
                itemResult[1] = 404
                return vDic; # not on every server
            itemResult[1] = 200
        itemResult[0] = 200
        if vDic['序號'].lower() != avID.lower():
            raise ValueError('序號 got and given not equal', vDic['序號'].lower(), avID.lower())
        dumpDic2Json(vDic)
        if showInfo:
            print("Downloaded ", avID)
    else:
        vDic = loadDicFromJson(avID)
        if itemResult[0] == 0:
            itemResult[0] = 200

    # check if img exist, otherwise download img
    if not checkAvIdImgExist(avID):
        downloadImage(vDic['封面圖'], avID)
        if showInfo:
            print("Img Downloaded", avID)
    
    return vDic


def dumpResult2Json(avIDResult):
    filename = 'avIDResult.json'
    f = open(filename, "w")
    json.dump(avIDResult, f)
    f.close()
    
def loadResultFromJson():
    filename = 'avIDResult.json'    
    f = open(filename, 'r')
    avIDResult = json.load(f)
    f.close()
    return avIDResult

def countAVResult(avIDResult):
    count0 = 0
    count404 = 0
    count200 = 0
    countWaitJavBus = 0
    for key, value in avIDResult.items():
        if value[0] == 0 and value[1] == 0:
            count0+=1
        elif value[0] == 200 or value[1] == 200:
            count200+=1
        elif value[0] == 404 and value[1] == 404:
            count404+=1
        elif value[0] == 404 and value[1] == 0:
            countWaitJavBus+=1
        else:
            raise ValueError("Unknow value combination: ", value[0], value[1])
    print("AVIDResult: 0~", count0, " 200~", count200, " 404~", count404, "wait trying JavBus~", countWaitJavBus)

from itertools import islice
from random import randint
import time
import random

    
def randomGet2(avIDResult, count):
    c_200 = 0
    c_404 = 0
    c_998 = 0
    c_999 = 0


    for i in range(count):
        key, value = random.choice(list(avIDResult.items()))

        t = 0
        print("{0: <10}".format(key), end='')
        if value[0] == 0 or (value[0] == 404 and value[1] == 0):    # not get from web before
            itemResult = avIDResult[key]
            vDic = completeAvID(key, itemResult)
            if len(vDic) > 0:
                if vDic['網站'] == 'Javhoo':
                    print("---- Jho downloaded, ", end='')
                elif vDic['網站'] == 'JavBus':
                    print("---- JBs downloaded, ", end='') 
                else:
                    raise ValueError("???? Where am I downloaded", vDic['網站'])
                c_200+=1
                t = random.uniform(0.5, 5.0)             
            else:
                c_404+=1
                print("@@@@ not on 2 Javs,  ", end='')
                t = random.uniform(0.5, 1.0)                   
        elif value[0] == 404 and value[1] == 404:
            print("#### 2 sites FAIL,   ", end='')
            c_998+=1
        else:
            print("!!!! already there,  ", end='')
            c_999+=1
        print("pause {0:.2f} seconds...".format(t))
        time.sleep(t)
    print("$$$$$ Batch for {0} items, {1} [200], {2} [404], {3} [exists], {4} [exists fail]."
          .format(count, c_200, c_404, c_999, c_998))


def serialGet(avIDResult, count):
    c_200 = 0
    c_404 = 0
    c_998 = 0
    c_999 = 0
    itemRemain = count

    for key, value in list(avIDResult.items()):
        t = 0
        if value[0] == 0 or (value[0] == 404 and value[1] == 0):    # not get from web before
            print("{0: <10}".format(key), end='')            
            itemResult = avIDResult[key]
            vDic = completeAvID(key, itemResult)
            if len(vDic) > 0:
                if vDic['網站'] == 'Javhoo':
                    print("---- Jho downloaded, ", end='')
                elif vDic['網站'] == 'JavBus':
                    print("---- JBs downloaded, ", end='') 
                else:
                    raise ValueError("???? Where am I downloaded", vDic['網站'])
                c_200+=1
                itemRemain-=1
                if (itemRemain == 0):
                    break
                t = random.uniform(0.5, 2.0)             
            else:
                c_404+=1
                print("@@@@ not on 2 Javs,  ", end='')
                t = random.uniform(0.1, 0.5)
            print("pause {0:.2f} seconds...".format(t))            
        elif value[0] == 404 and value[1] == 404:
#            print("{0: <10}#### 2 sites FAIL,   ".format(key), end='')
            c_998+=1
        else:
#            print("!!!! already there,  ", end='')
            c_999+=1
#        print("pause {0:.2f} seconds...".format(t))
        time.sleep(t)
    print("$$$$$ Batch for {0} items, {1} [200], {2} [404], {3} [exists], {4} [exists fail]."
          .format(count, c_200, c_404, c_999, c_998))

    
#another site :    https://www.arzon.jp/itemlist.html?t=&m=all&s=&q=JFD-07    這內容很好，但一開始有個同意頁，內容頁也較複雜
#also ano site:    https://javfree.me/?s=NTRD-037 
#another site :    http://www.javlib.com/cn/  這內容也很好，一開始也有頁，search好像不是從網址，要從網頁form
#another site :    https://onejav.com/

localFolder = '/Users/yiwenliu/Desktop/Projects/AV'

In [2]:
avIDResult = loadResultFromJson()

In [3]:
countAVResult(avIDResult)

AVIDResult: 0~ 176  200~ 4894  404~ 856 wait trying JavBus~ 6


In [7]:
start_time = time.time()
for i in range(1):
    print("Round: {0}".format(i))
    serialGet(avIDResult, 30)
    dumpResult2Json(avIDResult)
countAVResult(avIDResult)
elapsed_time = time.time() - start_time
print("Time consumed: {0:.2f} secs".format(elapsed_time))

Round: 0
PURE-004  ---- JBs downloaded, pause 1.48 seconds...
$$$$$ Batch for 30 items, 1 [200], 0 [404], 5045 [exists], 886 [exists fail].
AVIDResult: 0~ 0  200~ 5045  404~ 886 wait trying JavBus~ 1
Time consumed: 1.53 secs


In [ ]:
countAVResult(avIDResult)

In [ ]:
randomGet(avIDResult, 1)

In [ ]:
vDic = loadDicFromJson("CETD-230")

In [ ]:
vDic

In [ ]:
completeAvID("ABP-612")

In [ ]:
vDic = getDicFromJavBus("JMD-093")
print (vDic)

In [ ]:
vDic

In [ ]:
import os

total_files = []
countJH = 0
countJB_New = 0
countJH_New = 0
for root, dir, files in os.walk('/Users/yiwenliu/Desktop/Projects/AV'):
    for file in files:
        valueDic = {}
        if file.endswith("json"):
            fullname = os.path.join(root, file)
            f = open(fullname, 'r')
            valueDic = json.load(f)
            f.close()
            if '網站' in valueDic:
#                print ("ID {0} is downloaded from {1}".format(valueDic['序號'], valueDic['網站']))
                if valueDic['網站'] == 'Javhoo':
                    countJH_New+=1
                elif valueDic['網站'] == 'JavBus':
                    countJB_New+=1
                else:
                    raise ValueError("Unknow website", valueDic['網站'])
            else:
                valueDic['網站'] = 'Javhoo'
#                print (valueDic)
                print ("Add Javhoo to {0}".format(valueDic['序號']))
                countJH+=1
                f = open(fullname, "w")
                json.dump(valueDic, f)
                f.close()
print("Total files: {0}, Add Javhoo: {1}, Already Javhoo: {2}, Already Javbus: {3}".format(countJH+countJB_New+countJH_New, countJH, countJH_New, countJB_New))